In [1]:
import pandas as pd
import numpy as np

import re

from catboost import CatBoostClassifier
from xgboost import XGBClassifier

from tqdm.notebook import tqdm

import yaml
import joblib

In [2]:
def extract_purchases(string):
    return list(map(int, re.findall(r"'(\d+)'", string)))

In [3]:
config_path = "../config/params.yaml"
config = yaml.load(open(config_path), Loader=yaml.FullLoader)

preproc = config["preprocessing"]
recommender_train = config['train']['recommender']
win_predictor_train = config['train']['win_predictor']
evaluate = config['evaluate']

# Evaluate

В данном блоке мы получаем предсказания рекомендательной системы и модели, прогнозирующей победителя, на сырых данных для теста.  

In [5]:
df_train = pd.read_csv(preproc['train_data'])
df_train = df_train.set_index('index')
df_train[:5]

,purchase,forsmallbiz,price,customer,supplier,is_winner,vectorized,month,reg_code,purchase_size,flag_won,n_unique_okpd2
index,,,,,,,,,,,,
0,130546,1,184153.98,10530,52052,0,[ 0.15146377 0.30577086 0.03988786 0.041497...,11,31.0_86,2,0.0,18
1,128349,1,733000.00,10572,52052,0,[ 2.10939743e-02 3.61095411e-02 3.83576084e-...,2,22.2_86,2,0.0,18
2,129556,1,88125.00,10663,52052,0,[ 0.01750772 0.02390788 0.00398467 0.008696...,10,32.3_86,2,0.0,18
3,132850,1,36400.00,10733,52052,0,[ 1.39173070e-01 2.61834112e-01 -6.09590371e-...,6,21.2_86,2,0.0,18
4,130813,1,87000.00,10306,52052,0,[ 1.83479267e-02 1.59257488e-02 1.89327852e-...,7,23.1_86,3,0.0,18


In [4]:
df_evaluate = pd.read_csv(evaluate['evaluate_data'])
df_evaluate = df_evaluate.set_index('index')
df_evaluate[:5]

,purchase,region_code,min_publish_date,forsmallbiz,price,customer,okpd2_code,supplier,is_winner,vectorized
index,,,,,,,,,,
557804,253626,33,2022-11-14,1,34151.0,3389,drug,39153,1,[ 9.19683573e-02 2.03430614e-01 -4.18254503e-...
490364,166797,2,2021-09-23,1,43054.0,11558,21.2,56062,1,[ 3.30532570e-02 2.22863433e-02 1.08472357e-...
85366,30142,32,2021-09-20,0,94851.8,3018,22.2,14745,0,[ 5.21527714e-02 6.27693017e-02 -6.86741754e-...
886629,559462,37,2021-07-08,0,1398995.5,4747,42.9,17652,1,[ 0.12819854 -0.07514798 0.01455515 0.144088...
917602,590700,68,2022-10-18,0,230376.0,7868,drug,36900,0,[ 0.04999948 0.13985359 -0.03082386 0.013489...


In [6]:
df_submission = pd.read_csv(evaluate['recommend_sub_path'])
df_submission = df_submission.set_index('index')
df_submission[:5]

,purchases
index,
1,"[1729, 535051]"
10002,"[24722, 207646]"
1001,"[419951, 177230]"
10010,"[74211, 584516]"
100169,[466634]


In [7]:
df_submission = df_submission['purchases'].apply(extract_purchases)

In [8]:
# отсортируем по длине массива
df_submission = df_submission.reindex(df_submission.apply(len).sort_values(
    ascending=False).index)

In [8]:
def extract_words(string):
    return list(map(float, string[1:-1].split()))


def get_month(df_test):
    return df_test['min_publish_date'].apply(lambda x: int(x.split('-')[1]))


def add_reg_code(df_test):
    return df_test['okpd2_code'].astype('str') + '_' \
                 + df_test['region_code'].astype('str')


def add_purchase_size(df_test):
    return df_test.merge(df_test.groupby('purchase')['supplier'] \
                                .size().to_frame('purchase_size'), 
                         on='purchase', how='outer')

    
def add_flag(df_test, df_train):
    return df_test.merge(df_train[['supplier', 'customer', 'flag_won']] \
                                 .groupby(['supplier', 'customer']).tail(1), 
                         on=['supplier', 'customer'], how='left').fillna(0)


def add_unique_okpd2(df_test, df_train):
    return df_test.merge(df_train[['supplier', 'n_unique_okpd2']] \
                                 .groupby('supplier').tail(1), 
                         on=['supplier'], how='left').fillna(1)


def generate_features(df_test, df_train):
    """
    Генерация признаков
    """
    # Выделение месяца публикации закупки
    df_test['month'] = get_month(df_test)
    
    # Выделение инедентификатора, объединяющего регион и ОКПД закупки
    df_test['reg_code'] = add_reg_code(df_test)
    
    # Выделение кол-ва участников в закупке
    df_test = add_purchase_size(df_test)
    
    # Выделение флага работы поставщика с покупателем
    df_test = add_flag(df_test, df_train)
    
    # Выделение кол-ва уникальных ОКПД поставщика
    df_test = add_unique_okpd2(df_test, df_train)
    
    return df_test 

In [9]:
def pipeline_preproc(df_test: pd.DataFrame, df_train: pd.DataFrame) -> tuple:
    """
    Функция выполняет предобработку датасетов.

    Args:
        df_test (pd.DataFrame): тестовый датасет
        df_train (pd.DataFrame): тренировочный датасет

    Returns:
        tuple: кортеж из двух элементов, содержащий измененные тестовый и 
        тренировочный датасеты
    """

    # Выделение вектора токенов из строки
    df_train['vectorized'] = df_train['vectorized'].apply(extract_words)
    df_test['vectorized'] = df_test['vectorized'].apply(extract_words)
    
    # Генерация признаков
    df_test = generate_features(df_test, df_train)
    
    df_test = df_test.drop(columns=preproc['drop_columns'])
    
    # Преобразование типов столбцов
    df_train = df_train.astype(preproc['change_type_columns'])
    df_test = df_test.astype(preproc['change_type_columns'])

    
    return df_test, df_train

In [10]:
df_evaluate, df_train = pipeline_preproc(df_evaluate, df_train)

In [11]:
df_evaluate[:5]

,purchase,forsmallbiz,price,customer,supplier,is_winner,vectorized,month,reg_code,purchase_size,flag_won,n_unique_okpd2
0,253626,1,34151.0,3389,39153,1,"[0.0919683573, 0.203430614, -0.0418254503, 0.0...",11,drug_33,1,0.0,2
1,166797,1,43054.0,11558,56062,1,"[0.033053257, 0.0222863433, 0.00108472357, 0.0...",9,21.2_2,1,0.0,3
2,30142,0,94851.8,3018,14745,0,"[0.0521527714, 0.0627693017, -0.00686741754, 0...",9,22.2_32,1,0.0,8
3,559462,0,1398995.5,4747,17652,1,"[0.12819854, -0.07514798, 0.01455515, 0.144088...",7,42.9_37,1,0.0,3
4,590700,0,230376.0,7868,36900,0,"[0.04999948, 0.13985359, -0.03082386, 0.013489...",10,drug_68,1,0.0,1


## Recommender

In [ ]:
def transform_vector(df_train, df_test):
    """
    Генерация признаков на основе вектора токенов
    """
    for i in range(100):
        df_train[str(i)] = df_train['vectorized'].apply(lambda x: x[i])
        df_test[str(i)] = df_test['vectorized'].apply(lambda x: x[i])
        
    return df_train, df_test


def supplier_data(df_train: pd.DataFrame, df_test: pd.DataFrame, 
                  sup: str) -> tuple:
    """
    Filters train and test DataFrames based on the unique reg_code of a given supplier.
    Removes unnecessary columns and duplicates.
    Drops purchases that exist in both train and test DataFrames.
    
    Parameters:
    -----------
    df_train: pd.DataFrame
    The train DataFrame
    df_test: pd.DataFrame
    The test DataFrame
    sup: str
    The name of the supplier to filter the DataFrames by

    Returns:
    --------
    Tuple[pd.DataFrame, pd.DataFrame]
    A tuple of filtered DataFrames for train and test, respectively.
    """


    unique_reg_okpd = df_train[df_train['supplier'] == sup]['reg_code'].unique()
    
    # фильтруем train и test на основе уникальных reg_code поставщиков 
    df_sup_train = df_train[df_train['reg_code'].isin(unique_reg_okpd)]
    df_sup_test = df_test[df_test['reg_code'].isin(unique_reg_okpd)]

    
    if df_sup_test.empty:
        df_sup_test = df_test
    
    # удаляем ненужные для системы рекомендаций стобцы и дубликаты
    df_sup_train = df_sup_train.drop(columns=recommender_train['drop_columns']) \
                               .drop_duplicates()
    df_sup_test = df_sup_test.drop(columns=recommender_train['drop_columns']) \
                             .drop_duplicates()
    

    df_sup_test = df_sup_test.set_index('purchase')
    df_sup_train = df_sup_train.set_index('purchase')
    
    # удаляем закупки, которые есть и test, и в train
    df_sup_train = df_sup_train.drop(set(df_submission[sup]).intersection(df_sup_train.index))
    df_sup_test = df_sup_test[~df_sup_test.index.isin(df_sup_train.index)]
    
    
    return df_sup_train, df_sup_test   


def compute_recommends(df_train: pd.DataFrame, 
                       df_evaluate: pd.DataFrame, 
                       df_submission: pd.DataFrame, 
                       models: dict, n: int) -> pd.Series:
    """
    Определение рекомендаций для n наиболее активных пользователей
    """
    recommends = pd.Series()
    
    # Преобразование векторов
    df_train, df_evaluate = transform_vector(df_train, df_evaluate)
    
    # Итерация по каждому поставщику из набора df_submission
    for sup in tqdm(df_submission.index[:n]):
        
        # Получение набора данных для оценки для текущего поставщика
        df_sup_evaluate = supplier_data(df_train, df_evaluate, sup)[1]
        
        # Предсказание моделью
        y_pred = models[sup].predict(df_sup_evaluate)
        
        # Получение рекомендуемых товаров
        recommends[str(sup)] = df_sup_evaluate[y_pred==1].index.tolist()
    
    
    return recommends

In [ ]:
models = joblib.load(recommender_train['models'])

In [23]:
recommends = compute_recommends(df_train.copy(), df_evaluate.copy(), 
                                df_submission, models, 100)

/tmp/ipykernel_22299/931855057.py:70: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  recommends = pd.Series()
/tmp/ipykernel_22299/931855057.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_train[str(i)] = df_train['vectorized'].apply(lambda x: x[i])
/tmp/ipykernel_22299/931855057.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_test[str(i)] = df_test['vectorized'].apply(lambda x: x[i])
/tmp/ipykernel_2

  0%|          | 0/100 [00:00<?, ?it/s]

In [24]:
recommends[:5]

694     [588862, 101380, 160529, 152929, 152298, 51828...
5495    [430622, 433589, 605138, 595582, 430410, 42893...
2468    [211473, 601367, 604033, 34831, 76616, 101291,...
12      [115672, 587178, 167221, 146021, 20005, 78767,...
2622    [99074, 136902, 410088, 596704, 34842, 158353,...
dtype: object

## Probability of winning

In [25]:
df_evaluate[:5]

,purchase,forsmallbiz,price,customer,supplier,vectorized,is_winner,month,reg_code,purchase_size,flag_won,n_unique_okpd2
0,596065,0,53874.00,9582,2940,"[0.12289912, 0.24247403, -0.06769952, -0.00474...",1,3,drug_77,1,1.0,2
1,467821,1,7500000.00,9591,7538,"[0.05361487, 0.01407206, -0.00556184, -0.02367...",1,9,38.2_77,1,0.0,4
2,88928,1,281370.35,7831,11016,"[0.07224658, 0.01950507, 0.01632774, 0.0257298...",0,8,26.2_68,3,0.0,5
3,88928,1,281370.35,7831,574,"[0.07224658, 0.01950507, 0.01632774, 0.0257298...",0,8,26.2_68,3,0.0,8
4,88928,1,281370.35,7831,7749,"[0.07224658, 0.01950507, 0.01632774, 0.0257298...",0,8,26.2_68,3,0.0,8


In [28]:
def remove_columns(df_test):
    df_test = df_test.drop(columns=win_predictor_train['drop_columns'])
    
    return df_test.drop('is_winner', axis=1)

def get_meta_features(x_test: pd.DataFrame, 
                       catboost: CatBoostClassifier, 
                       xgboost: XGBClassifier) -> np.ndarray:
    """
    Функция для получения мета-признаков на тестовых данных.

    Аргументы:
    - x_test: pd.DataFrame, тестовые данные, на которых будет делаться предсказание.
    - catboost: CatBoostClassifier, обученная модель CatBoost.
    - xgboost: XGBClassifier, обученная модель XGBoost.

    Возвращает:
    - predictions_test: np.ndarray, массив значений предсказанных классов для каждого объекта тестовых данных.
    """

    # Предсказание вероятности класса 1 на тестовых данных с помощью CatBoost
    y_score = catboost.predict_proba(x_test)
    predictions_test = y_score[:, 1]
    
    # Предсказание вероятности класса 1 на тестовых данных с помощью XGBoost
    y_score = xgboost.predict_proba(x_test)
    predictions_test = np.vstack((predictions_test, y_score[:, 1]))
    
    # Транспонирование массива и возврат его значения
    return predictions_test.T


def compute_winner(df_evaluate: pd.DataFrame, 
                    models: dict) -> np.ndarray:
    """
    Функция для получения прогноза победителя в торгах на основе мета-признаков.

    Аргументы:
    - df_evaluate: pd.DataFrame, данные для оценки, на основе которых будет делаться прогноз.
    - models: dict, словарь, содержащий обученные модели для прогнозирования.

    Возвращает:
    - np.ndarray, массив, содержащий вероятности того, что поставщик победит в торгах.
    """

    # Удаление столбцов, которые не нужны для предсказания
    df_evaluate = remove_columns(df_evaluate)
    
    # Получение мета-признаков с помощью функции get_meta_features
    meta_features = get_meta_features(df_evaluate, 
                                      models['catboost'], 
                                      models['xgboost'])
    
    # Предсказание вероятности класса 1 на мета-признаках с помощью Naive Bayes
    return models['Naive_bayes'].predict_proba(meta_features)[:, 1]

In [29]:
models = joblib.load(win_predictor_train['models'])

In [30]:
compute_winner(df_evaluate, models)[:10]

array([0.99999998, 0.98055527, 0.35806481, ..., 0.99999451, 0.99848522,
       0.99992103])